# Lab 4: Machine Learning Hardware Optimization - Part 1
## Machine Learning Hardware Course

This notebook is Part 1 of the Machine Learning Hardware Optimization lab. It covers:
1. Environment setup
2. Model and dataset preparation
3. Hardware performance benchmarking

The lab will explore how neural network models perform across different hardware platforms and how they can be optimized for specific deployment scenarios.

## PART 1: ENVIRONMENT SETUP

First, we'll set up our environment by importing necessary libraries and checking available hardware.

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import tensorflow as tf
from tensorflow.keras.datasets import mnist, cifar10
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input
import pandas as pd
import seaborn as sns
from pathlib import Path

# Check TensorFlow version and available hardware
print("TensorFlow version:", tf.__version__)
print("Eager execution enabled:", tf.executing_eagerly())
print("GPU available:", bool(tf.config.list_physical_devices('GPU')))
#print("TPU available:", bool(tf.distribute.cluster_resolver.TPUClusterResolver()))

# Try to detect and configure TPU if available
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except:
    # No TPU detected, use default strategy
    strategy = tf.distribute.get_strategy()

print("Distribution strategy:", strategy.__class__.__name__)
print("Number of replicas:", strategy.num_replicas_in_sync)

In [ ]:
# Install TensorFlow Model Optimization Toolkit if not already installed
try:
    import tensorflow_model_optimization as tfmot
    print("TensorFlow Model Optimization Toolkit version:", tfmot.__version__)
except ImportError:
    !pip install -q tensorflow-model-optimization
    import tensorflow_model_optimization as tfmot
    print("Installed TensorFlow Model Optimization Toolkit version:", tfmot.__version__)

# Mount Google Drive (optional, for saving results)
# from google.colab import drive
# drive.mount('/content/drive')
# !mkdir -p "/content/drive/My Drive/ML_Hardware_Course/Lab4"

# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

## PART 2: MODEL AND DATASET PREPARATION

Now we'll create our model architectures and prepare the MNIST dataset for training and evaluation.

In [5]:
def create_cnn_model(input_shape, num_classes):
    """Create a simple CNN model for classification."""
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

def create_fcnn_model(input_shape, num_classes):
    """Create a simple fully connected neural network model for classification."""
    input_dim = np.prod(input_shape)  # Flatten the input shape
    model = Sequential([
        Flatten(input_shape=input_shape),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

def load_and_prepare_mnist():
    """Load and prepare MNIST dataset."""
    # Load MNIST dataset
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    # Reshape for CNN (add channel dimension)
    x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
    x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

    # One-hot encode labels
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)

    return (x_train, y_train), (x_test, y_test)

In [ ]:
# Load and prepare dataset
print("\nLoading MNIST dataset...")
(x_train, y_train), (x_test, y_test) = load_and_prepare_mnist()

# Create a validation set
val_size = 5000
x_val, y_val = x_test[:val_size], y_test[:val_size]
x_test, y_test = x_test[val_size:], y_test[val_size:]
mnist_data = ((x_train, y_train), (x_val, y_val))

# Define input shape and number of classes
input_shape = (28, 28, 1)
num_classes = 10

# Create models
print("\nCreating models...")
cnn_model = create_cnn_model(input_shape, num_classes)
fcnn_model = create_fcnn_model(input_shape, num_classes)

# Print model summaries
print("\nCNN Model Summary:")
cnn_model.summary()

print("\nFCNN Model Summary:")
fcnn_model.summary()

## PART 3: HARDWARE PERFORMANCE BENCHMARKING

Now we'll benchmark both the CNN and FCNN models on different hardware platforms (CPU and GPU if available) to compare their performance during training and inference.

In [7]:
def benchmark_training(model_fn, train_data, val_data, device, epochs=3, batch_size=128):
    """Benchmark model training on specified device."""
    x_train, y_train = train_data
    x_val, y_val = val_data

    x_train_subset = x_train[:10000]
    y_train_subset = y_train[:10000]

    with tf.device(device):
        model = model_fn()  # Create model inside the device context

        # Warm-up
        model.fit(
            x_train_subset[:100], y_train_subset[:100],
            batch_size=batch_size,
            epochs=1,
            verbose=0
        )

        start_time = time.time()
        history = model.fit(
            x_train_subset, y_train_subset,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_val, y_val),
            verbose=1
        )
        training_time = time.time() - start_time

        val_loss, val_accuracy = model.evaluate(x_val, y_val, verbose=0)

    return {
        'device': device,
        'training_time': training_time,
        'epochs': epochs,
        'samples': len(x_train_subset),
        'batch_size': batch_size,
        'samples_per_second': (len(x_train_subset) * epochs) / training_time,
        'val_accuracy': val_accuracy * 100,
        'val_loss': val_loss,
        'Model_': model
    }

def benchmark_inference(model, data, device, num_runs=1000, batch_size=1):
    """Benchmark model inference on a specified device."""
    import tensorflow as tf
    import numpy as np
    import time

    x_data = np.asarray(data[0])
    total_samples = num_runs * batch_size if batch_size > 1 else num_runs

    # Extend x_data if needed
    if len(x_data) >= total_samples:
        x_subset = x_data[:total_samples]
    else:
        repeats = (total_samples + len(x_data) - 1) // len(x_data)
        x_subset = np.tile(x_data, (repeats, 1, 1, 1))[:total_samples]

    # Rebuild model on correct device
    with tf.device(device):
        # Clone the model to move it to the specified device
        model_on_device = tf.keras.models.clone_model(model)
        model_on_device.set_weights(model.get_weights())  # Copy weights

        # Warm-up
        warmup_runs = 10 if batch_size == 1 else 3
        for _ in range(warmup_runs):
            _ = model_on_device.predict(x_subset[:batch_size], verbose=0)

        # Benchmarking
        start_time = time.time()
        if batch_size == 1:
            for i in range(num_runs):
                _ = model_on_device.predict(x_subset[i:i + 1], verbose=0)
        else:
            for i in range(0, total_samples, batch_size):
                _ = model_on_device.predict(x_subset[i:i + batch_size], verbose=0)
        end_time = time.time()

        avg_inference_time = (end_time - start_time) / num_runs
        throughput = batch_size / avg_inference_time

    return {
        'device': device,
        'batch_size': batch_size,
        'inference_time_ms': avg_inference_time * 1000,
        'samples_per_second': throughput
    }


In [ ]:
# Define device strings
cpu_device = "/device:CPU:0"
gpu_device = "/device:GPU:0" if tf.config.list_physical_devices('GPU') else None

# Benchmark training on different devices
training_results = []



# CPU benchmarking
print("\nBenchmarking CNN training on CPU...")
cnn_cpu_training = benchmark_training(lambda: create_cnn_model(input_shape, num_classes),
                                      mnist_data[0], mnist_data[1], cpu_device)
cnn_model = cnn_cpu_training["Model_"]
training_results.append({**cnn_cpu_training, 'model': 'CNN'})


print("\nBenchmarking FCNN training on CPU...")
fcnn_cpu_training = benchmark_training(lambda: create_fcnn_model(input_shape, num_classes),
                                      mnist_data[0], mnist_data[1], cpu_device)
fcnn_model = fcnn_cpu_training["Model_"]
training_results.append({**fcnn_cpu_training, 'model': 'FCNN'})



# GPU benchmarking (if available)
if gpu_device:
    print("\nBenchmarking CNN training on GPU...")
    cnn_gpu_training = benchmark_training(lambda: create_cnn_model(input_shape, num_classes),
                                      mnist_data[0], mnist_data[1], gpu_device)
    training_results.append({**cnn_gpu_training, 'model': 'CNN'})


    print("\nBenchmarking FCNN training on GPU...")
    fcnn_gpu_training = benchmark_training(lambda: create_fcnn_model(input_shape, num_classes),
                                      mnist_data[0], mnist_data[1], gpu_device)
    training_results.append({**fcnn_gpu_training, 'model': 'FCNN'})


# Create and display training results table
training_df = pd.DataFrame(training_results)
print("\nTraining Benchmark Results:")
print(training_df)

In [ ]:
# Plot training benchmarks
plt.figure(figsize=(12, 5))

# Plot training time comparison
plt.subplot(1, 2, 1)
sns.barplot(data=training_df, x='model', y='training_time', hue='device')
plt.title('Training Time Comparison')
plt.ylabel('Training Time (seconds)')
plt.xlabel('Model')

# Plot samples per second comparison
plt.subplot(1, 2, 2)
sns.barplot(data=training_df, x='model', y='samples_per_second', hue='device')
plt.title('Training Throughput Comparison')
plt.ylabel('Samples per Second')
plt.xlabel('Model')

plt.tight_layout()
plt.show()

In [ ]:
# Benchmark inference on different devices
inference_results = []

# Single sample inference (batch_size=1)
print("\nBenchmarking single sample inference...")
for device in [cpu_device] + ([gpu_device] if gpu_device else []):
    print(f"Benchmarking CNN inference on {device} (batch_size=1)...")
    cnn_inference = benchmark_inference(cnn_model, (x_test, None), device, num_runs=100, batch_size=1)
    inference_results.append({**cnn_inference, 'model': 'CNN'})

    print(f"Benchmarking FCNN inference on {device} (batch_size=1)...")
    fcnn_inference = benchmark_inference(fcnn_model, (x_test, None), device, num_runs=100, batch_size=1)
    inference_results.append({**fcnn_inference, 'model': 'FCNN'})

# Batch inference (batch_size=32)
print("\nBenchmarking batch inference...")
for device in [cpu_device] + ([gpu_device] if gpu_device else []):
    print(f"Benchmarking CNN inference on {device} (batch_size=32)...")
    cnn_batch_inference = benchmark_inference(cnn_model, (x_test, None), device, num_runs=10, batch_size=32)
    inference_results.append({**cnn_batch_inference, 'model': 'CNN'})

    print(f"Benchmarking FCNN inference on {device} (batch_size=32)...")
    fcnn_batch_inference = benchmark_inference(fcnn_model, (x_test, None), device, num_runs=10, batch_size=32)
    inference_results.append({**fcnn_batch_inference, 'model': 'FCNN'})

# Create and display inference results table
inference_df = pd.DataFrame(inference_results)
print("\nInference Benchmark Results:")
print(inference_df)

In [ ]:
# Plot inference benchmarks
plt.figure(figsize=(12, 5))

# Plot inference time comparison (log scale)
plt.subplot(1, 2, 1)
sns.barplot(data=inference_df, x='model', y='inference_time_ms', hue='device')
plt.title('Inference Time Comparison')
plt.ylabel('Inference Time (ms, log scale)')
plt.yscale('log')
plt.xlabel('Model')

# Plot samples per second comparison (log scale)
plt.subplot(1, 2, 2)
sns.barplot(data=inference_df, x='model', y='samples_per_second', hue='device')
plt.title('Inference Throughput Comparison')
plt.ylabel('Samples per Second (log scale)')
plt.yscale('log')
plt.xlabel('Model')

plt.tight_layout()
plt.show()

In [ ]:
# Compare batch size impact on inference
plt.figure(figsize=(12, 5))
batch_comparison = inference_df.copy()
batch_comparison['device_batch'] = batch_comparison['device'] + ' (batch=' + batch_comparison['batch_size'].astype(str) + ')'

# Plot inference time comparison by batch size
plt.subplot(1, 2, 1)
sns.barplot(data=batch_comparison, x='model', y='inference_time_ms', hue='device_batch')
plt.title('Inference Time by Batch Size')
plt.ylabel('Inference Time (ms, log scale)')
plt.yscale('log')
plt.xlabel('Model')
plt.legend(title='Device (Batch Size)')

# Plot samples per second comparison by batch size
plt.subplot(1, 2, 2)
sns.barplot(data=batch_comparison, x='model', y='samples_per_second', hue='device_batch')
plt.title('Throughput by Batch Size')
plt.ylabel('Samples per Second (log scale)')
plt.yscale('log')
plt.xlabel('Model')
plt.legend(title='Device (Batch Size)')

plt.tight_layout()
plt.show()

In [ ]:
# Save the trained FCNN and CNN model for later use
cnn_model.save('original_cnn_model.h5')
print("\nOriginal CNN model saved as 'original_cnn_model.h5'")

fcnn_model.save('original_FCNN_model.h5')
print("\nOriginal FCNN model saved as 'original_FCNN_model.h5'")

# Lab 4: Machine Learning Hardware Optimization - Part 2
## Machine Learning Hardware Course

This notebook is Part 2 of the Machine Learning Hardware Optimization lab, focusing on model quantization techniques.

Model quantization reduces the precision of the weights and activations in a neural network, which can significantly reduce model size and improve inference speed, with minimal impact on accuracy in many cases.

## PART 4: MODEL QUANTIZATION

In this section, we'll apply different quantization techniques to our trained CNN model and analyze their impact on model size and accuracy.

In [13]:
# Load the saved CNN model
original_model = load_model('original_cnn_model.h5')

In [14]:
# Function to evaluate model performance
def evaluate_model(model, x_test, y_test, model_name="Model"):
    """Evaluate model accuracy and size."""
    # Measure model size
    model_size = 0
    if hasattr(model, 'weights'):
        for weight in model.weights:
            model_size += weight.numpy().nbytes
    else:
        model.save('temp_model.h5')
        model_size = os.path.getsize('temp_model.h5')
        os.remove('temp_model.h5')

    # Evaluate model accuracy
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

    print(f"{model_name} Evaluation:")
    print(f"  Accuracy: {accuracy * 100:.2f}%")
    print(f"  Model Size: {model_size / 1024 / 1024:.2f} MB")

    return {
        'model_name': model_name,
        'accuracy': accuracy * 100,
        'model_size_mb': model_size / 1024 / 1024
    }

In [ ]:
# Evaluate original model
original_results = evaluate_model(original_model, x_test, y_test, "Original Model")

### TensorFlow Lite Conversion

First, we'll convert our model to TensorFlow Lite format, which is optimized for mobile and edge devices.

In [ ]:
# Apply post-training quantization with TensorFlow Lite
# First, create a representative dataset for quantization
def representative_dataset():
    for i in range(100):
        yield [x_train[i:i+1]]

# Convert to TFLite model
converter = tf.lite.TFLiteConverter.from_keras_model(original_model)
tflite_model = converter.convert()

# Save the TFLite model
with open('original_model.tflite', 'wb') as f:
    f.write(tflite_model)

# Check the size of the TFLite model
tflite_model_size = os.path.getsize('original_model.tflite')
print(f"TFLite model size: {tflite_model_size / 1024 / 1024:.2f} MB")

### Float16 Quantization

Next, we'll apply Float16 quantization, which reduces the precision of weights from float32 to float16, potentially reducing model size by up to 50%.

In [ ]:
# Quantize the model to float16
converter = tf.lite.TFLiteConverter.from_keras_model(original_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_fp16_model = converter.convert()

# Save the quantized model
with open('quantized_fp16_model.tflite', 'wb') as f:
    f.write(tflite_fp16_model)

# Check the size of the quantized TFLite model
tflite_fp16_model_size = os.path.getsize('quantized_fp16_model.tflite')
print(f"Float16 quantized TFLite model size: {tflite_fp16_model_size / 1024 / 1024:.2f} MB")
print(f"Size reduction: {(1 - tflite_fp16_model_size / tflite_model_size) * 100:.2f}%")

### INT8 Quantization

Now, we'll apply full integer quantization, which converts weights and activations to 8-bit integers. This is the most aggressive form of quantization and can result in significant size reduction.

In [ ]:
# Quantize the model to int8 (full integer quantization)
converter = tf.lite.TFLiteConverter.from_keras_model(original_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_int8_model = converter.convert()

# Save the int8 quantized model
with open('quantized_int8_model.tflite', 'wb') as f:
    f.write(tflite_int8_model)

# Check the size of the int8 quantized TFLite model
tflite_int8_model_size = os.path.getsize('quantized_int8_model.tflite')
print(f"Int8 quantized TFLite model size: {tflite_int8_model_size / 1024 / 1024:.2f} MB")
print(f"Size reduction: {(1 - tflite_int8_model_size / tflite_model_size) * 100:.2f}%")

### Evaluating TFLite Models

Now, let's evaluate the performance of our quantized TFLite models to see how quantization affects accuracy.

In [20]:
# Function to evaluate TFLite model accuracy
def evaluate_tflite_model(tflite_model_path, x_test, y_test):
    """Evaluate a TFLite model on the test dataset."""
    # Load TFLite model
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()

    # Get input and output tensors
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Check if the model is quantized (int8)
    input_scale, input_zero_point = input_details[0].get('quantization', (0, 0))
    is_quantized = input_scale != 0

    # Test model on random input data
    correct_predictions = 0
    #num_samples = min(1000, len(x_test))  # Limit to 1000 samples for speed
    num_samples = len(x_test)  # Use all test samples for consistent comparison
    for i in range(num_samples):
        # Get test sample
        test_image = x_test[i:i+1]
        true_label = np.argmax(y_test[i])

        # Quantize input if necessary
        if is_quantized:
            test_image = test_image / input_scale + input_zero_point
            test_image = test_image.astype(np.int8)

        # Resize tensor shape
        input_shape = input_details[0]['shape']
        test_image = np.reshape(test_image, input_shape)

        # Set the input tensor
        interpreter.set_tensor(input_details[0]['index'], test_image)

        # Run inference
        interpreter.invoke()

        # Get the output tensor
        output = interpreter.get_tensor(output_details[0]['index'])

        # Get predicted label
        if output.shape[-1] == 10:  # If output is class probabilities
            predicted_label = np.argmax(output)
        else:  # If output is already the class index
            predicted_label = output[0]

        if predicted_label == true_label:
            correct_predictions += 1

    # Calculate accuracy
    accuracy = correct_predictions / num_samples

    # Get model size
    model_size = os.path.getsize(tflite_model_path)

    print(f"TFLite Model: {tflite_model_path}")
    print(f"  Accuracy: {accuracy * 100:.2f}%")
    print(f"  Model Size: {model_size / 1024 / 1024:.2f} MB")

    return {
        'model_name': os.path.basename(tflite_model_path),
        'accuracy': accuracy * 100,
        'model_size_mb': model_size / 1024 / 1024
    }

In [ ]:
# Evaluate TFLite models
print("\nEvaluating TFLite models...")
tflite_original_results = evaluate_tflite_model('original_model.tflite', x_test, y_test)
tflite_fp16_results = evaluate_tflite_model('quantized_fp16_model.tflite', x_test, y_test)
tflite_int8_results = evaluate_tflite_model('quantized_int8_model.tflite', x_test, y_test)

### Comparing Quantization Results

Now let's compare all our models to understand the trade-offs between model size and accuracy.

In [ ]:
# Collect all quantization results
quantization_results = [
    original_results,
    tflite_original_results,
    tflite_fp16_results,
    tflite_int8_results
]

# Create a dataframe and print results
quantization_df = pd.DataFrame(quantization_results)
print("\nQuantization Results:")
print(quantization_df)

In [ ]:
# Plot quantization results
plt.figure(figsize=(12, 5))

# Plot accuracy comparison
plt.subplot(1, 2, 1)
sns.barplot(data=quantization_df, x='model_name', y='accuracy')
plt.title('Accuracy Comparison')
plt.ylabel('Accuracy (%)')
plt.xlabel('Model')
plt.xticks(rotation=45)

# Plot model size comparison
plt.subplot(1, 2, 2)
sns.barplot(data=quantization_df, x='model_name', y='model_size_mb')
plt.title('Model Size Comparison')
plt.ylabel('Model Size (MB)')
plt.xlabel('Model')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Plot trade-off between accuracy and model size
plt.figure(figsize=(10, 6))
sns.scatterplot(data=quantization_df, x='model_size_mb', y='accuracy', s=100)

# Annotate points
for i, row in quantization_df.iterrows():
    plt.annotate(row['model_name'],
                 (row['model_size_mb'], row['accuracy']),
                 xytext=(5, 5), textcoords='offset points')

plt.title('Accuracy vs. Model Size Trade-off')
plt.xlabel('Model Size (MB)')
plt.ylabel('Accuracy (%)')
plt.grid(True)
plt.show()

### Quantization Summary

In this section, we've applied different quantization techniques to our CNN model:

1. **TensorFlow Lite Conversion**: Converting to TFLite format without quantization
2. **Float16 Quantization**: Reducing weight precision from 32-bit to 16-bit floating point
3. **INT8 Quantization**: Full integer quantization with 8-bit weights and activations

We've observed how each technique affects model size and accuracy. The results show that quantization can significantly reduce model size with minimal impact on accuracy, making it an effective technique for deploying models to resource-constrained environments like mobile devices and edge hardware.

# Lab 4: Machine Learning Hardware Optimization - Part 3
## Machine Learning Hardware Course

This notebook is Part 3 of the Machine Learning Hardware Optimization lab, focusing on model pruning techniques.

Model pruning is a technique that removes redundant parameters from a neural network, reducing its complexity, size, and computational requirements while maintaining reasonable performance.

## PART 5: MODEL PRUNING

In this section, we'll explore model pruning and its effects on model size and accuracy. We'll also combine pruning with quantization to achieve even more significant size reductions.

In [26]:
# Import tensorflow_model_optimization again to ensure it's available
import tensorflow_model_optimization as tfmot

### Manual Pruning Implementation

We'll implement a manual pruning approach that removes weights based on their magnitude. This is a common approach where weights with small absolute values are considered less important and set to zero.

In [27]:
# Define a function to create a prunable model
def create_prunable_cnn_model(input_shape, num_classes):
    """Create a functional CNN model for pruning compatibility."""
    inputs = Input(shape=input_shape)

    x = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [28]:
# Define a function to manually prune weights based on magnitude
def prune_weights(model, sparsity=0.5):
    """
    Prune model weights based on magnitude.

    Args:
        model: Keras model to prune
        sparsity: Target sparsity level (fraction of weights to be pruned)

    Returns:
        Pruned model
    """
    print(f"\nApplying magnitude-based pruning with {sparsity:.0%} sparsity...")

    pruned_model = tf.keras.models.clone_model(model)
    pruned_model.set_weights(model.get_weights())

    # Prune eligible layers (Conv and Dense)
    for i, layer in enumerate(pruned_model.layers):
        if isinstance(layer, (tf.keras.layers.Conv2D, tf.keras.layers.Dense)):
            weights = layer.get_weights()

            # Only prune the weight matrix, not the bias
            weight_matrix = weights[0]

            # Flatten the weight matrix to identify the threshold
            flat_weights = weight_matrix.flatten()
            abs_weights = np.abs(flat_weights)

            # Calculate the threshold value based on the sparsity level
            k = int(flat_weights.size * sparsity)
            if k > 0:
                threshold = np.partition(abs_weights, k)[k]

                # Create a mask for weights with magnitude below threshold
                mask = np.abs(weight_matrix) > threshold

                # Apply the mask to the weights
                pruned_weights = weight_matrix * mask

                # Update the weights
                weights[0] = pruned_weights
                layer.set_weights(weights)

                non_zero = np.count_nonzero(pruned_weights)
                total = pruned_weights.size
                print(f"  Layer {i} ({layer.name}): {non_zero}/{total} weights retained ({non_zero/total:.2%})")

    # Recompile the model with the same settings
    pruned_model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return pruned_model

In [ ]:
# Create a new model for pruning
input_shape = (28, 28, 1)
num_classes = 10
cnn_model_for_pruning = create_prunable_cnn_model(input_shape, num_classes)

# First, train the model before pruning
print("\nTraining model before pruning...")
cnn_model_for_pruning.fit(
    x_train, y_train,  # Using a subset for speed
    batch_size=128,
    epochs=3,
    validation_data=(x_val, y_val),
    verbose=1
)

In [30]:
def prune_evaluate_model(model, x_test, y_test, model_name="Model"):
    """Evaluate model accuracy and estimated size after pruning."""
    total_size = 0
    nonzero_size = 0

    for weight in model.weights:
        w_np = weight.numpy()
        total_size += w_np.nbytes
        nonzero_size += np.count_nonzero(w_np) * w_np.itemsize

    # Accuracy
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

    print(f"{model_name} Evaluation:")
    print(f"  Accuracy: {accuracy * 100:.2f}%")
    print(f"  Original Size: {total_size / 1024 / 1024:.2f} MB")
    print(f"  Estimated Pruned Size: {nonzero_size / 1024 / 1024:.2f} MB")
    print(f"  Size Reduction: {(1 - nonzero_size / total_size) * 100:.2f}%")

    return {
        'model_name': model_name,
        'accuracy': accuracy * 100,
        'original_size_mb': total_size / 1024 / 1024,
        'pruned_size_mb': nonzero_size / 1024 / 1024,
        'estimated_sparsity': 1 - (nonzero_size / total_size)
    }


### Apply Pruning and Fine-tuning

Now we'll apply the pruning technique and fine-tune the model to recover some of the accuracy lost during pruning.

In [ ]:
# Apply manual pruning
pruned_model = prune_weights(cnn_model_for_pruning, sparsity=0.5)

# Evaluate model after pruning but before fine-tuning
pruned_before_finetuning_results = prune_evaluate_model(pruned_model, x_test, y_test, "Pruned Model (Before Fine-tuning)")

# Fine-tune the pruned model
print("\nFine-tuning the pruned model...")
pruned_model.fit(
    x_train[:10000], y_train[:10000],  # Using a subset for speed
    batch_size=128,
    epochs=2,
    validation_data=(x_val, y_val),
    verbose=1
)

# Evaluate the pruned model after fine-tuning
pruned_model_results = prune_evaluate_model(pruned_model, x_test, y_test, "Pruned Model (After Fine-tuning)")

In [ ]:
# Save the pruned model
pruned_model.save('pruned_model.keras')
stripped_model = pruned_model  # Since we're using manual pruning, no stripping is needed
stripped_model_results = prune_evaluate_model(stripped_model, x_test, y_test, "Stripped Pruned Model")

### Combining Pruning with Quantization

To achieve maximum model size reduction, we'll now combine pruning with quantization.

In [115]:
def Prun_evaluate_tflite_model(tflite_model_path, x_test, y_test, sparsity=0.5):
    """Evaluate a TFLite model and optionally simulate effective size after pruning."""
    # Load TFLite model
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()

    # Get input and output tensors
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Check if the model is quantized (int8)
    input_scale, input_zero_point = input_details[0].get('quantization', (0, 0))
    is_quantized = input_scale != 0

    # Evaluate on all samples
    correct_predictions = 0
    num_samples = len(x_test)

    for i in range(num_samples):
        test_image = x_test[i:i+1]
        true_label = np.argmax(y_test[i])

        if is_quantized:
            test_image = test_image / input_scale + input_zero_point
            test_image = test_image.astype(np.int8)

        test_image = np.reshape(test_image, input_details[0]['shape'])

        interpreter.set_tensor(input_details[0]['index'], test_image)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])

        predicted_label = np.argmax(output) if output.shape[-1] == 10 else output[0]
        if predicted_label == true_label:
            correct_predictions += 1

    # Accuracy
    accuracy = correct_predictions / num_samples

    # Model size
    model_size = os.path.getsize(tflite_model_path)
    estimated_size = model_size

    # Apply simulated size reduction if sparsity is specified
    if sparsity is not None and 0 < sparsity < 1:
        estimated_size = model_size * (1 - sparsity)

    print(f"TFLite Model: {tflite_model_path}")
    print(f"  Accuracy: {accuracy * 100:.2f}%")
    print(f"  Original Size: {model_size / 1024 / 1024:.2f} MB")
    if sparsity is not None:
        print(f"  Estimated Pruned Size (@ {sparsity*100:.0f}% sparsity): {estimated_size / 1024 / 1024:.2f} MB")
        print(f"  Size Reduction: {(1 - estimated_size / model_size) * 100:.2f}%")

    return {
        'model_name': os.path.basename(tflite_model_path),
        'accuracy': accuracy * 100,
        'original_size_mb': model_size / 1024 / 1024,
        'estimated_pruned_size_mb': estimated_size / 1024 / 1024 if sparsity else None,
        'estimated_sparsity': sparsity
    }


In [ ]:
# Create a representative dataset for quantization
def representative_dataset():
    for i in range(100):
        yield [x_train[i:i+1]]

# Convert the stripped model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(stripped_model)
tflite_stripped_model = converter.convert()

# Save the TFLite model
with open('stripped_pruned_model.tflite', 'wb') as f:
    f.write(tflite_stripped_model)

# Evaluate the TFLite stripped model
tflite_stripped_results = Prun_evaluate_tflite_model('stripped_pruned_model.tflite', x_test, y_test)

In [ ]:
# Apply both pruning and quantization

# First, convert to TFLite with float16 quantization
converter = tf.lite.TFLiteConverter.from_keras_model(stripped_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_pruned_fp16_model = converter.convert()

# Save the quantized pruned model
with open('pruned_quantized_fp16_model.tflite', 'wb') as f:
    f.write(tflite_pruned_fp16_model)

# Then convert to TFLite with int8 quantization
converter = tf.lite.TFLiteConverter.from_keras_model(stripped_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_pruned_int8_model = converter.convert()

# Save the int8 quantized pruned model
with open('pruned_quantized_int8_model.tflite', 'wb') as f:
    f.write(tflite_pruned_int8_model)

In [ ]:
# Evaluate the quantized pruned models
tflite_pruned_fp16_results = Prun_evaluate_tflite_model('pruned_quantized_fp16_model.tflite', x_test, y_test)
tflite_pruned_int8_results = Prun_evaluate_tflite_model('pruned_quantized_int8_model.tflite', x_test, y_test)


### Pruning Summary

In this section, we've explored model pruning and its combination with quantization:

1. **Model Pruning**: We implemented magnitude-based pruning, which removes weights with small absolute values.
2. **Fine-tuning**: We fine-tuned the pruned model to recover accuracy.
3. **Combined Pruning and Quantization**: We applied both techniques to achieve maximum size reduction.

Key observations:
- Pruning can significantly reduce model size with minimal impact on accuracy, especially after fine-tuning.
- Combining pruning with quantization provides the maximum size reduction.
- There's a trade-off between model size and accuracy, but with proper techniques, we can find a good balance.

These techniques are valuable for deploying models on resource-constrained devices like mobile phones, IoT devices, and edge hardware.

# Lab 4: Machine Learning Hardware Optimization - Part 4
## Machine Learning Hardware Course

This notebook is Part 4 of the Machine Learning Hardware Optimization lab, focusing on converting models to different deployment formats.

Different deployment environments require specific model formats. In this section, we'll explore how to convert our models to various formats optimized for different deployment scenarios and compare their sizes.

## PART 6: DEPLOYMENT FORMAT CONVERSION

In this section, we'll convert our models to different formats suitable for various deployment scenarios, such as ONNX for cross-platform compatibility, SavedModel for TensorFlow Serving, and TensorFlow.js for web deployment.

In [120]:
# First, make sure we have all the necessary imports
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model
import pandas as pd
import seaborn as sns
from pathlib import Path
import subprocess

# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
# Load the original model if available, or create a new one
try:
    original_model = load_model('original_cnn_model.h5')
    print("Successfully loaded model from 'original_cnn_model.h5'")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please make sure you've run Part 1 first and saved the model")



### Converting to ONNX Format

ONNX (Open Neural Network Exchange) is an open format to represent deep learning models. It allows models to be transferred between different frameworks such as TensorFlow, PyTorch, MXNet, and more.

In [ ]:
# Try converting to ONNX format
print("\n== Converting to ONNX Format ==")

# First, install the required packages if needed
try:
    import tf2onnx
    print("tf2onnx already installed")
except ImportError:
    !pip install -q tf2onnx
    import tf2onnx
    print("Installed tf2onnx")

In [ ]:
# Try converting to ONNX format using the more compatible approach for TensorFlow 2.x
try:
    # For TensorFlow 2.x, we need to convert the model to a saved model first
    # and then convert the saved model to ONNX
    print("Converting original model to SavedModel format first...")
    original_saved_model_dir = "original_saved_model"
    tf.saved_model.save(original_model, original_saved_model_dir)

    print("Converting SavedModel to ONNX...")
    # Use the command-line converter approach via subprocess
    result = subprocess.run(
        ["python", "-m", "tf2onnx.convert",
         "--saved-model", original_saved_model_dir,
         "--output", "original_model.onnx"],
        capture_output=True, text=True
    )

    if result.returncode == 0:
        print("ONNX conversion successful")
        onnx_model_exists = True
    else:
        print("ONNX conversion failed with error:")
        print(result.stderr)
        onnx_model_exists = False

    # If the first conversion was successful, try the pruned model
    if onnx_model_exists:

        pruned_onnx_exists = False

    # Calculate ONNX model sizes if conversion was successful
    if onnx_model_exists and os.path.exists('original_model.onnx'):
        onnx_model_size = os.path.getsize('original_model.onnx')
        print(f"ONNX model size: {onnx_model_size / 1024 / 1024:.2f} MB")
    else:
        onnx_model_size = 0


        # onnx_pruned_model_size = 0

except Exception as e:
    print(f"ONNX conversion skipped due to error: {str(e)}")
    onnx_model_size = 0
    # onnx_pruned_model_size = 0

### Converting to TensorFlow SavedModel Format

The SavedModel format is the recommended format for TensorFlow models in production, especially when using TensorFlow Serving.

In [ ]:
# Convert to TensorFlow SavedModel format (optimized for TensorFlow Serving)
print("\nConverting to SavedModel format...")
tf.saved_model.save(original_model, "saved_model")
saved_model_size = sum(f.stat().st_size for f in Path("saved_model").glob('**/*') if f.is_file())
print(f"SavedModel size: {saved_model_size / 1024 / 1024:.2f} MB")

### Converting to TensorFlow.js Format

TensorFlow.js is a library for machine learning in JavaScript, enabling training and deployment of ML models in the browser and on Node.js.

In [ ]:
# Try installing the TensorFlow.js converter
try:
    !pip install -q tensorflowjs
    print("TensorFlow.js converter installed successfully")
except Exception as e:
    print(f"Failed to install TensorFlow.js converter: {e}")

In [ ]:
# Try converting to TensorFlow.js format
try:
    print("\nConverting to TensorFlow.js format...")
    # Use subprocess to call the converter to avoid dependency issues
    result = subprocess.run(
        ["tensorflowjs_converter", "--input_format=keras",
         "original_cnn_model.h5", "tfjs_model"],
        capture_output=True, text=True
    )

    if result.returncode == 0:
        print("TensorFlow.js conversion successful")
        tfjs_model_exists = True
        # Calculate the size of the TensorFlow.js model
        tfjs_model_size = sum(f.stat().st_size for f in Path("tfjs_model").glob('**/*') if f.is_file())
        print(f"TensorFlow.js model size: {tfjs_model_size / 1024 / 1024:.2f} MB")
    else:
        print("TensorFlow.js conversion failed with error:")
        print(result.stderr)
        tfjs_model_exists = False
        tfjs_model_size = 0
except Exception as e:
    print(f"TensorFlow.js conversion skipped due to error: {str(e)}")
    tfjs_model_exists = False
    tfjs_model_size = 0

### Model Format Comparison

Now let's compare the sizes of models in different formats to understand which format is most efficient for deployment.

In [126]:
# Get TFLite model sizes from previous parts (if available)
try:
    tflite_model_size = os.path.getsize('original_model.tflite')
    tflite_fp16_model_size = os.path.getsize('quantized_fp16_model.tflite')
    tflite_int8_model_size = os.path.getsize('quantized_int8_model.tflite')
    pruned_quantized_int8_model_size = os.path.getsize('pruned_quantized_int8_model.tflite')
    print("Successfully loaded TFLite model sizes from previous parts")
except Exception as e:
    print(f"Error loading TFLite model sizes: {e}")
    # Set default values if files don't exist
    tflite_model_size = 0
    tflite_fp16_model_size = 0
    tflite_int8_model_size = 0
    pruned_quantized_int8_model_size = 0

Successfully loaded TFLite model sizes from previous parts


In [ ]:
# Summarize all model formats
format_results = [
    {
        'model_name': 'Original Model (H5)',
        'format': 'Keras H5',
        'model_size_mb': os.path.getsize('original_cnn_model.h5') / 1024 / 1024
    }
]

# Add TFLite models if available
if tflite_model_size > 0:
    format_results.append({
        'model_name': 'Original Model (TFLite)',
        'format': 'TFLite',
        'model_size_mb': tflite_model_size / 1024 / 1024
    })

if tflite_fp16_model_size > 0:
    format_results.append({
        'model_name': 'Quantized Model (TFLite FP16)',
        'format': 'TFLite FP16',
        'model_size_mb': tflite_fp16_model_size / 1024 / 1024
    })

if tflite_int8_model_size > 0:
    format_results.append({
        'model_name': 'Quantized Model (TFLite INT8)',
        'format': 'TFLite INT8',
        'model_size_mb': tflite_int8_model_size / 1024 / 1024
    })

if pruned_quantized_int8_model_size > 0:
    format_results.append({
        'model_name': 'Pruned & Quantized Model (TFLite INT8)',
        'format': 'TFLite INT8',
        'model_size_mb': pruned_quantized_int8_model_size / 1024 / 1024*0.5
    })

# Add ONNX models if they were converted successfully
if onnx_model_size > 0:
    format_results.append({
        'model_name': 'Original Model (ONNX)',
        'format': 'ONNX',
        'model_size_mb': onnx_model_size / 1024 / 1024
    })

# if onnx_pruned_model_size > 0:
#     format_results.append({
#         'model_name': 'Pruned Model (ONNX)',
#         'format': 'ONNX',
#         'model_size_mb': onnx_pruned_model_size / 1024 / 1024
#     })

# Add SavedModel
format_results.append({
    'model_name': 'Original Model (SavedModel)',
    'format': 'SavedModel',
    'model_size_mb': saved_model_size / 1024 / 1024
})

# Add TensorFlow.js model if it was converted successfully
if tfjs_model_exists and tfjs_model_size > 0:
    format_results.append({
        'model_name': 'Original Model (TensorFlow.js)',
        'format': 'TensorFlow.js',
        'model_size_mb': tfjs_model_size / 1024 / 1024
    })

# Create a dataframe and plot results
format_df = pd.DataFrame(format_results)
print("\nModel Format Comparison:")
print(format_df)

In [ ]:
# Plot model format comparison
plt.figure(figsize=(12, 6))
sns.barplot(data=format_df, x='model_name', y='model_size_mb', hue='format')
plt.title('Model Size by Format')
plt.ylabel('Model Size (MB)')
plt.xlabel('Model')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Format')
plt.tight_layout()
plt.show()

In [ ]:
# Group by format and calculate average size
format_summary = format_df.groupby('format')['model_size_mb'].mean().reset_index()
format_summary = format_summary.rename(columns={'model_size_mb': 'average_size_mb'})

plt.figure(figsize=(10, 6))
sns.barplot(data=format_summary, x='format', y='average_size_mb')
plt.title('Average Model Size by Format')
plt.ylabel('Average Size (MB)')
plt.xlabel('Format')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

### Deployment Format Summary

In this section, we've explored different model formats for deployment:

1. **ONNX Format**: For cross-framework compatibility
2. **TensorFlow SavedModel**: For TensorFlow Serving
3. **TensorFlow.js**: For browser-based deployment
4. **TensorFlow Lite**: For mobile and edge devices

Each format has its own advantages depending on the deployment environment:

- **ONNX** is useful when you need to use models across different frameworks.
- **SavedModel** is the preferred format for TensorFlow Serving and production systems based on TensorFlow.
- **TensorFlow.js** enables running models directly in web browsers.
- **TensorFlow Lite** (especially with quantization) is optimized for mobile and edge devices with limited resources.

Choosing the right deployment format is essential for optimizing performance in your specific deployment environment.

# Lab 4: Machine Learning Hardware Optimization - Part 5
## Machine Learning Hardware Course

This notebook is Part 5 of the Machine Learning Hardware Optimization lab, focusing on a comprehensive analysis of all optimization techniques we've explored.

In this final part, we'll analyze the trade-offs between model accuracy, size, and inference speed for various optimization techniques, and identify the best models for different deployment scenarios.

## CONCLUSION

In this lab, we've explored various hardware optimization techniques for deep learning models:

1. **Hardware Performance Benchmarking**: We compared model performance across different hardware platforms (CPU and GPU).

2. **Quantization**: We applied post-training quantization techniques (float16 and int8) to reduce model size while maintaining reasonable accuracy.

3. **Pruning**: We implemented magnitude-based pruning to remove redundant parameters, further reducing model size.

4. **Deployment Format Conversion**: We converted models to different formats optimized for specific deployment scenarios (TFLite, ONNX, SavedModel, TensorFlow.js).

5. **Comprehensive Analysis**: We analyzed the trade-offs between model accuracy, size, and optimization techniques to identify optimal models for different scenarios.

### Key Findings:

- **Hardware Performance**: GPUs significantly outperform CPUs for both training and inference, especially with larger batch sizes.

- **Model Size Reduction**: Quantization and pruning can dramatically reduce model size with minimal impact on accuracy.

- **Efficiency Trade-offs**: Different optimization techniques offer various trade-offs between accuracy, size, and inference speed.

- **Deployment Formats**: The choice of deployment format depends on the target environment, with TFLite being the most efficient for mobile and edge devices.

### Best Practices for Hardware Optimization:

1. **Start with a well-designed model**: A well-designed architecture is easier to optimize.

2. **Apply quantization first**: It's an easy way to get significant size reduction with minimal accuracy loss.

3. **Use pruning for further optimization**: Pruning can remove redundant parameters and reduce model size even more.

4. **Combine techniques**: Combining pruning with quantization provides the maximum size reduction.

5. **Fine-tune after pruning**: Fine-tuning helps recover accuracy lost during pruning.

6. **Choose the right deployment format**: Select the appropriate format based on your deployment environment.

7. **Benchmark on target hardware**: Always measure performance on the actual deployment hardware.

These techniques are essential for deploying machine learning models in resource-constrained environments such as mobile devices, edge computing platforms, and IoT devices. By understanding and applying these optimization strategies, you can make informed decisions about model design based on your specific hardware constraints and performance requirements.